<a href="https://colab.research.google.com/github/sunnatillo91/ai-datascience/blob/main/3_1_df_to_sql_and__create_update_delete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science va Sun'iy Intellekt Praktikum
- 3-MODUL Ma'lumotlarga ishlov berish
- 3.1 Fayllar bilan ishlash

In [2]:
import pandas as pd
import sqlite3 as sql

In [3]:
db = sql.connect('/content/drive/MyDrive/praktikum_datasets/newdb.db')
cursor = db.cursor()

In [36]:
df = pd.read_csv('/content/drive/MyDrive/praktikum_datasets/Electricity_Production_By_Source.csv')
df.head()

,Entity,Code,Year,Electricity from coal (TWh),Electricity from gas (TWh),Electricity from hydro (TWh),Electricity from other renewables (TWh),Electricity from solar (TWh),Electricity from oil (TWh),Electricity from wind (TWh),Electricity from nuclear (TWh)
0,Afghanistan,AFG,2000,0.0,0.155,0.312,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,2001,0.0,0.094,0.498,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,2002,0.0,0.132,0.555,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,AFG,2003,0.0,0.309,0.630,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,AFG,2004,0.0,0.329,0.565,0.0,0.0,0.0,0.0,0.0


MO (Database) bilan ishlashda oson bo'lishi uchun ustunlarga qisqa nomlar beramiz

In [5]:
cols = ['country','code','year','coal','gas','hydro','renewables','solar','oil','wind','nuclear']
df.columns = cols

In [6]:
df.head(3)

,country,code,year,coal,gas,hydro,renewables,solar,oil,wind,nuclear
0,Afghanistan,AFG,2000,0.0,0.155,0.312,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,AFG,2001,0.0,0.094,0.498,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,AFG,2002,0.0,0.132,0.555,0.0,0.0,0.0,0.0,0.0


In [7]:
# df.to_sql('elect',db)

In [7]:
query = "SELECT * FROM elect WHERE country='Uzbekistan'"
df = pd.read_sql(query, db)
df.head(3)

,index,country,code,year,coal,gas,hydro,renewables,solar,oil,wind,nuclear
0,5964,Uzbekistan,UZB,2000,11.602405,25.523451,5.817,0.0,0.0,1.380143,0.0,0.0
1,5965,Uzbekistan,UZB,2001,11.766320,25.884038,5.899,0.0,0.0,1.399642,0.0,0.0
2,5966,Uzbekistan,UZB,2002,12.214374,26.869686,6.124,0.0,0.0,1.452939,0.0,0.0


# Yangi jadval yaratish

In [8]:
query = """
CREATE TABLE IF NOT EXISTS worldpop(
  rank INT,
  country TEXT PRIMARY KEY,
  density DOUBLE,
  population DOUBLE NOT NULL,
  area INT NOT NULL
);
"""
cursor.execute(query)
db.commit()

In [9]:
query = "SELECT name FROM sqlite_master WHERE type='table'"
cursor.execute(query)
cursor.fetchall()

[('elect',), ('worldpop',)]

In [10]:
query = "PRAGMA TABLE_INFO(worldpop)"
cursor.execute(query)
cursor.fetchall()

[(0, 'rank', 'INT', 0, None, 0),
 (1, 'country', 'TEXT', 0, None, 1),
 (2, 'density', 'DOUBLE', 0, None, 0),
 (3, 'population', 'DOUBLE', 1, None, 0),
 (4, 'area', 'INT', 1, None, 0)]

# Jadvalga yangi ma'lumotlar qo'shish

In [11]:
query = "INSERT INTO worldpop VALUES (1, 'Macau', 21946.4667, 658.3940, 30)"
cursor.execute(query)
db.commit()

IntegrityError: UNIQUE constraint failed: worldpop.country

In [24]:
cursor.execute("SELECT * FROM worldpop")
cursor.fetchall()

[]

# Faqat kerakli ustunlarga qiymat berish

In [25]:
query = "INSERT INTO worldpop (country, population, area) VALUES ('Uzbekistan', 33935.7630, 447400)"
cursor.execute(query)
db.commit()

In [26]:
cursor.execute("SELECT * FROM worldpop")
cursor.fetchall()

[(None, 'Uzbekistan', None, 33935.763, 447400)]

# MO table to df

In [27]:
pd.read_sql("SELECT * FROM worldpop", db)

,rank,country,density,population,area
0,None,Uzbekistan,None,33935.763,447400


In [28]:
query = "UPDATE worldpop SET rank=133, density=75.8511 WHERE country='Uzbekistan'"
cursor.execute(query)
db.commit()

In [29]:
cursor.execute("SELECT * FROM worldpop")
cursor.fetchall()

[(133, 'Uzbekistan', 75.8511, 33935.763, 447400)]

# MO Jadvaldagi keraksiz qatorni o'chirish

In [18]:
query = "DELETE FROM worldpop WHERE country='Macau'"
cursor.execute(query)
db.commit()

In [19]:
cursor.execute("SELECT * FROM worldpop")
cursor.fetchall()

[(133, 'Uzbekistan', 75.8511, 33935.763, 447400)]

In [33]:
df = pd.read_sql("SELECT * FROM worldpop", db)
df

,rank,country,density,population,area
0,133,Uzbekistan,75.8511,33935.763,447400


# MO dagi keraksiz jadvalni o'chirish

In [20]:
cursor.execute("DROP TABLE worldpop")
db.commit()

In [21]:
cursor.execute("SELECT * FROM worldpop")  # worldpop jadvali o'chirilgani sabab xato qaytadi
cursor.fetchall()

OperationalError: no such table: worldpop

In [39]:
df = pd.read_csv('/content/drive/MyDrive/praktikum_datasets/world_population.csv')
df.head()

,rank,country,density,densityMi,pop2021,area
0,1,Macau,21946.4667,56841.3487,658.394,30
1,2,Monaco,19755.5000,51166.7450,39.511,2
2,3,Singapore,8305.1915,21510.4461,5896.686,710
3,4,Hong Kong,6841.3134,17719.0017,7552.810,1104
4,5,Gibraltar,5616.3333,14546.3033,33.698,6


In [37]:
query = """
CREATE TABLE IF NOT EXISTS worldpop(
  rank INT,
  country TEXT PRIMARY KEY,
  density DOUBLE,
  population DOUBLE NOT NULL,
  area INT NOT NULL
);
"""
cursor.execute(query)
db.commit()

In [40]:
uz = df[df.country=='Uzbekistan']
uz

,rank,country,density,densityMi,pop2021,area
132,133,Uzbekistan,75.8511,196.4542,33935.763,447400


In [ ]:
rank = uz.index.values[0]
country = uz.at[rank, 'country']
density = uz.at[rank, 'density']
population = uz.at[rank, 'pop2021']
area = uz.at[rank, 'area']

query = f"""
INSERT INTO worldpop VALUES ({rank}, '{country}', {density}, {population}, {area})
"""

In [54]:
mon = df[df.country=='Monaco']
mon

,rank,country,density,densityMi,pop2021,area
1,2,Monaco,19755.5,51166.745,39.511,2


In [57]:
rank = mon.index.values[0]
country = mon.at[rank, 'country']
density = mon.at[rank, 'density']
population = mon.at[rank, 'pop2021']
area = mon.at[rank, 'area']

query = f"""
INSERT INTO worldpop VALUES ({rank}, '{country}', {density}, {population}, {area})
"""
cursor.execute(query)
db.commit()

In [58]:
cursor.execute("SELECT * FROM worldpop")
cursor.fetchall()

[(133, 'Uzbekistan', 75.8511, 33935.763, 447400),
 (1, 'Monaco', 19755.5, 39.511, 2)]

In [59]:
db.close()